 <h1><center> Food Delivery in Manhattan </center></h1>

## 1. Problem Definition 

A food delivery company has decided to open a new office in Manhattan. They hired a data scientist in order to find the most suitable location of this new office, in which the shipment costs are minimized and the demand of food delivery is high.
This project will explore where the highest concentration of restaurants is located. This minimizes the shipment costs (since we would have an higher number of venues close to our new office) and improves the productivity, since the single shipment requires less time and more shipments can be done in the same time interval.
We will also look at the highest average score of each cluster of restaurants, since each costumer is likely to order from an higher rated restaurant (in terms of 
value for money).

## 2. Data 


The data necessary to accomplish my Data Science Task are the one of FourSquare. Using an API call, I will retrieve a Json file containing all the venues and convert it into a Pandas dataframe. Those data already contains position of the restaurants and their scores, as well as the number of reviews. Then I will create several clusters depending on the area that the Food Delivery company is willing to cover, and evaluate the best location to place the new office based on the FourSquare data (as described in the problem definition).